In [475]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import  HashingVectorizer
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
import jieba
import re
# 初始化参数
df = pd.read_excel('demo.xlsx',sheet_name='demo',usecols=[0,1])
meta = pd.read_excel('demo.xlsx',sheet_name='meta',usecols=[0,1],dtype=str)

# code  = pd.read_excel('D:\python\pytorch-learn\code.xlsx',sheet_name='基础编码',usecols=[0,1,2,3,4])
df['code'] = df['code'].apply(lambda x: x.split('-'))
all_labels = [item for sublist in df['code'] for item in sublist]
label_counts = Counter(all_labels)
rare_labels = {label for label, count in label_counts.items() if count == 1}
jieba.add_word("1.5米")
jieba.add_word("1米")
jieba.add_word("2米")
jieba.add_word("1.8米")
jieba.add_word("线")
jieba.add_word("100w")
jieba.add_word("88w")
jieba.add_word("66w")
jieba.add_word("67w")
jieba.add_word("80w")
jieba.add_word("200w")
jieba.add_word("60w")
jieba.add_word("45w")
jieba.add_word("cc")
jieba.add_word("120w")
jieba.add_word("88w")
jieba.add_word("荣max")
jieba.add_word("荣")
jieba.add_word("max")
jieba.add_word("30w")
jieba.add_word("65w")
jieba.add_word("op")
jieba.add_word("oppo")
jieba.add_word("mi")
jieba.add_word("mini")
jieba.add_word("mirco")
jieba.add_word("vo")
jieba.add_word("v")
jieba.add_word("240w")
jieba.add_word("pro")
#  '14' '15' '150' '150w' '1516' '16' '18w' '1米' '2' '200w' '20w' '22.5' '240w' '25w' '29w' '29wtypec' '2a' '2米' '3' '30w' '33w' '35w' '4' '40w'
jieba.add_word("150w")
jieba.add_word("22.5w")
jieba.add_word("18w")
jieba.add_word("30w")
jieba.add_word("33w")
jieba.add_word("35w")
jieba.add_word("40w")
jieba.add_word("29w")
jieba.add_word("typec")
jieba.add_word("tpc")
jieba.add_word("大壳头")
jieba.add_word("头")
jieba.add_word("10a")
jieba.add_word("6a")
jieba.add_word("5a")
jieba.add_word("85w")
jieba.add_word("max")
jieba.add_word("usb")
jieba.add_word("d")
jieba.add_word("g")
jieba.add_word("plus")
jieba.add_word("18w")
jieba.add_word("12w")
jieba.add_word("55w")
jieba.add_word("90w")
jieba.add_word("55w")
jieba.add_word("44w")
jieba.add_word("10w")
jieba.add_word("快充头")
jieba.add_word("快充")
jieba.add_word("闪充头")
jieba.add_word("闪充")
jieba.add_word("安卓线")
jieba.add_word("安卓")
jieba.add_word("华")
jieba.add_word("双口")
jieba.add_word("c口")
jieba.add_word("单口")
jieba.add_word("华为",freq=0)
jieba.add_word("ac")
jieba.add_word("vvivo",freq=0)
stop_words=["【", "】", "+","（","）","(",")"," ","*",",","/","=","|"]
#stop_words=[]
def chinese_tokenizer(context):
# 使用正则表达式去除自定义停止词
    for stop_word in stop_words:
        context = context.replace(stop_word, ' ')
    context = context.replace('-', '')
    context = context.replace('vivo', 'vvivo')
    context = context.lower()
    return list(jieba.cut(context))


vectorizer = CountVectorizer(
    analyzer=chinese_tokenizer,
    token_pattern=None,
    lowercase=True,
    binary=False
)

md = MultiOutputClassifier(DecisionTreeClassifier(criterion="entropy",random_state=42,max_depth=None,max_features=None))
pipeline = Pipeline([
    ('tfidf', vectorizer),
    ('clf', md)
])


# 创建一个新的 DataFrame 来保存稀有类别的样本
df_rare = df[df['code'].apply(lambda x: any(item in rare_labels for item in x))]

# 从原始 DataFrame 中移除这些稀有类别的样本
df_common = df.drop(df_rare.index)
mlb = MultiLabelBinarizer()
meta['code'] = meta['code'].apply(lambda x:[x])
mlb.fit(meta['code'])
#mlb.fit(df_common['code'])
Y_common = mlb.transform(df_common['code'])
# 拆分常见类别的数据集
X_train_common, X_test_common, Y_train_common, Y_test_common = train_test_split(
    df_common['name'], Y_common, test_size=0.20, random_state=45
)
# 将稀有类别的样本添加到训练集中
X_train_rare = df_rare['name']
Y_train_rare = mlb.transform(df_rare['code'])
# print(X_train_rare.shape, Y_train_rare.shape)
# 合并训练集
X_train = pd.concat([meta['name'],X_train_common, X_train_rare], ignore_index=True)
Y_train = np.vstack([mlb.transform(meta['code']),Y_train_common, Y_train_rare])
print(X_train.shape, Y_train.shape)
# 测试集保持不变
X_test = X_test_common
Y_test = Y_test_common
#print(mlb.classes_)
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)
print(pipeline.score(X_train, Y_train))
print(pipeline.score(X_test, Y_test))

# 打印特征名称（词汇表）
# print("Feature names:", vectorizer.get_feature_names_out())
print(classification_report(Y_test, Y_pred, target_names=mlb.classes_))
X_pre = ["3星线【c-c】25W双C线【2米线】"]
n_pred = pipeline.predict(X_pre)
predicted_codes = mlb.inverse_transform(n_pred)
# print(cross_val_score(pipeline, X_train, Y_train, cv=5))
for name, codes in zip(X_pre, predicted_codes):
   print(f"Product Name: {name} -> Predicted Codes: {', '.join(codes)}")



(2499,) (2499, 166)
0.9983993597438976
0.8502581755593803
              precision    recall  f1-score   support

  1010010001       0.78      0.88      0.82         8
  1010010002       0.50      1.00      0.67         1
  1010010003       1.00      1.00      1.00         2
  1010020001       0.83      0.71      0.77         7
  1010020002       0.75      1.00      0.86         3
  1010020003       1.00      1.00      1.00         2
  1010150001       1.00      0.75      0.86         8
  1010150002       0.50      0.75      0.60         4
  1010150003       0.00      0.00      0.00         0
  1020010004       1.00      0.83      0.91         6
  1020010005       1.00      1.00      1.00         6
  1020010006       1.00      1.00      1.00         4
  1020020004       1.00      1.00      1.00         2
  1020020005       1.00      1.00      1.00         5
  1020020006       0.86      1.00      0.92         6
  1020150004       0.00      0.00      0.00         0
  1020150005       1.00

d:\miniconda3\envs\s\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\miniconda3\envs\s\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\miniconda3\envs\s\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\miniconda3\envs\s\Lib\site-packages\sklearn\metric

In [ ]:
df

In [183]:
import jieba
jieba.add_word("华为",freq=1)
list(jieba.cut("华为【66W快充头+2米线】",HMM=True,cut_all=True))

['华为', '【', '66W', '快充', '快充头', '+', '2米', '米线', '】']

In [376]:
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import jieba
# 初始化参数
df = pd.read_excel('demo.xlsx',sheet_name='demo',usecols=[0,1,2,3])

df['code'] = df['code'].apply(lambda x: x.split(','))
all_labels = [item for sublist in df['code'] for item in sublist]
label_counts = Counter(all_labels)
rare_labels = {label for label, count in label_counts.items() if count == 1}
# 稀有类别对验证没有任何帮助，直接去除
df['code'] = df['code'].apply(lambda x: np.unique( [item for item in x if item not in rare_labels]))
# rare_labels,np.unique(df['code'].values)
df = df[df['code'].apply(lambda x: len(x)>0)]
# 构建特征, 使用l2进行归一化，smooth_idf 方式分母为0
tf = TfidfVectorizer(norm="l2",smooth_idf=True)
df['feature'] = df[['series', 'type', 'name']].astype(str).apply(' '.join, axis=1)
print(df['feature'].head(5))
X = tf.fit_transform(df['feature'])
mlb = MultiLabelBinarizer()

y = mlb.fit_transform(df['code'])

# 方差过滤
from sklearn.feature_selection import VarianceThreshold

X_var = VarianceThreshold().fit_transform(X)


# 方差太小，说明特征无效
X_pd = pd.DataFrame(X.toarray())
X_pd.var().mean(),X_pd.var().median(),X_pd.var().min(),X_pd.var().max()

0     苹果 数据线 主推数据线苹果15编织线1.5米线(一条)
1          华 数据线 主推数据线华为快充线1米线(一条)
2          华 数据线 主推数据线华为快充线1米线(两条)
3        华 数据线 主推数据线华为快充线1.5米线(一条)
4    安卓 数据线 主推数据线安卓梯形micro线2米线(两条)
Name: feature, dtype: object


(np.float64(0.0005911994271138112),
 np.float64(0.00024186554835175925),
 np.float64(3.6901704552422166e-05),
 np.float64(0.02073848176224028))

In [ ]:
# 构建特征, 使用l2进行归一化，smooth_idf 方式分母为0
import jieba
tf = TfidfVectorizer(norm="l2",smooth_idf=True,analyzer=jieba.cut)
X = tf.fit_transform(df['name'])



# 方差过滤
from sklearn.feature_selection import VarianceThreshold

X_var = VarianceThreshold().fit_transform(X)


# 方差变大，说明区别明显了
X_pd = pd.DataFrame(X.toarray())
X_pd.var().mean(),X_pd.var().median(),X_pd.var().min(),X_pd.var().max()

(np.float64(0.001927978523826236),
 np.float64(0.000798134963955713),
 np.float64(4.452107091598113e-05),
 np.float64(0.023728835386231224))

In [3]:
# 尝试直接count， jieba 分词
import jieba
from sklearn.feature_extraction.text import  CountVectorizer
stop=["【", "】", "+","（","）","(",")"," ","*",",","/","=","|"]
tf = CountVectorizer(analyzer=jieba.cut)
X = tf.fit_transform(df['name'])



# 方差过滤
from sklearn.feature_selection import VarianceThreshold

X_var = VarianceThreshold().fit_transform(X)


# 方差变大，说明区别明显了
X_pd = pd.DataFrame(X.toarray())
X_pd.var().mean(),X_pd.var().median(),X_pd.var().min(),X_pd.var().max()

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.500 seconds.
Prefix dict has been built successfully.


(np.float64(0.01752685805104584),
 np.float64(0.001810359126497556),
 np.float64(0.0002266545784224851),
 np.float64(0.49361364371154876))

In [519]:
# 交叉验证
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import cross_val_score,KFold
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC




# 假设你有 X (特征), y (标签)
skf = MultilabelStratifiedKFold(n_splits=5,shuffle=True,random_state=42)

md = MultiOutputClassifier(DecisionTreeClassifier(criterion="log_loss",max_depth=10,random_state=42))

# 使用 cross_val_score 并指定 scoring 参数 X_embedded X_var
cross_val_score(md,X_embedded,y,cv=skf,scoring="accuracy").mean()

np.float64(0.8900606542733553)

In [ ]:
# 尝试结果
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import cross_val_score,KFold
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

# 假设你有 X (特征), y (标签)
skf = MultilabelStratifiedKFold(n_splits=5)

md = MultiOutputClassifier(DecisionTreeClassifier(criterion="log_loss",splitter="random"))

demo_df = pd.read_excel("D:/work/采购/现实仓库/demo_.xlsx",usecols=["商品简称"])
demo_df["商品简称"] = demo_df["商品简称"].str.replace("\t","")

md.fit(X_var,y)

X_input = vectorizer.transform(demo_df["商品简称"].values.ravel())
n_pred = md.predict(X_input)
predicted_codes = mlb.inverse_transform(n_pred)
re = []
for code in predicted_codes:
    s = ",".join(code)
    re.append(s)
demo_df['预测结果'] = re
demo_df.to_excel("D:/work/采购/现实仓库/demo_.xlsx")


In [512]:
# 尝试直接count
import jieba
from sklearn.feature_extraction.text import  CountVectorizer
# 尝试清理噪音 ,"",""
stop=["【", "】", "+","（","）","(",")","*",",","/","=","|",";","以上","如果","邮费","\t","单条装",
      "一条","两条装","其他","，","一个","三条","超级","开盖","公对公","快充不伤机","不含线","2件","4件",
      "1条装","2条装","1条","2条","3条","条","两条","单条","没有","光速","加长","充电器套装","充电器","显示","是","笔记本电脑","优惠装","搭配装","优惠","链接","同长","默认","爆款","秒","备注","品牌"
      ,"发","冲头","两","电竞版","请","充电数据线","安卓快充电线","快充电线","充电线","闪充线","闪充数据线","看后面","后面","普通","单个头","快充头","快充线","正品","专用头","专用"
      ,"2个","1个","3个","2根","3根","6根","单个","单头","闪充头","闪充","快充","线长","通用","极速","系列","适配器","配","套装"]
jieba.load_userdict("userdict.txt")
jieba.del_word("口红")
jieba.del_word("头套")
def chinese_tokenizer(context):
    # 使用正则表达式去除自定义停止词
    context = context.lower()
    for stop_word in stop:
        context = context.replace(stop_word, ' ')
    context = context.replace('-', '')
    context = context.replace('iapd', 'ipad')
    context = context.replace('编制', '编织')
    context = context.replace('华为', '华')
    context = context.replace('平果', '苹果')
    context = context.replace('viov', 'vo')
    context = context.replace('vovi', 'vo')
    context = context.replace('oppo', 'op')
    context = context.replace('双tpc', '双c')
    context = context.replace('双typec', '双c')
    context = context.replace('绿口线', '绿口')
    context = context.replace('紫口线', '紫口')
    context = context.replace('白口线', '白口')
    context = context.replace('tpc口', 'c口')
    context = context.replace('白色', '白')
    context = context.replace('白头', '白')
    context = context.replace('小白', '白')
    context = context.replace('看后面', '后面')
    context = context.replace('米6a线', '6a')
    context = context.replace('米6a', '6a')
    context = context.replace('华总', '华')
    context = context.replace('高攻', 'g')
    context = context.replace('高功', 'g')
    context = context.replace('高', 'g')
    context = context.replace('sam星', '三星')
    re =  list(jieba.cut(context,cut_all=False,HMM=False,use_paddle=True))
    if any(word == "鲨" for word in re):
        print(context)
    return re


vectorizer = CountVectorizer(
    analyzer=chinese_tokenizer,
    token_pattern=None,
    lowercase=True,
    binary=True
)
X = vectorizer.fit_transform(df['feature'])

print(vectorizer.get_feature_names_out())
# 方差过滤
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

# pca = PCA(n_components=0.97,svd_solver="full")  # 降维到2维，白化

# X_dr = pca.fit_transform(X.toarray())  # 训练
# print(X_dr.shape)  # 降维后的数据
# # 去除噪音
# X = pca.inverse_transform(X_dr)  # 还原数据
# print(X.shape)  # 还原后的数据

X_var = X

[' ' '.' '1' '1.5米' '1.8米' '10' '100' '100w' '10a' '10w' '11' '120' '120w'
 '12a' '12w' '14' '15' '150' '150w' '16' '168' '18w' '1米' '2' '200w' '20w'
 '22.5w' '240w' '25w' '29w' '2a' '2米' '3' '30w' '33' '33w' '35w' '3t' '3星'
 '3米' '4' '40' '40w' '44w' '45' '45w' '4a' '5' '55w' '5a' '5t' '5v2a'
 '5v4a' '6' '60w' '614' '61w' '65' '65w' '66' '66w' '67' '67w' '6a' '6t'
 '7' '8' '80' '80w' '814' '85w' '87w' '88' '88w' '8a' '9' '90' '90w'
 '91619' '96w' 'a' 'a1181' 'a1278' 'a1286' 'a1343' 'a1369' 'a1370' 'a1398'
 'a14' 'a1424' 'a1425' 'a1465' 'a1502' 'a1534' 'a1706' 'a1707' 'a1708'
 'a1932' 'a1990' 'a21' 'a2141' 'a2363' 'a2518' 'c' 'cc' 'cro' 'ctoc'
 'ctoc口' 'c口' 'd' 'e' 'findx6' 'findx8' 'g' 'h' 'ic' 'ipad' 'iqoo' 'k10'
 'k11' 'k11x' 'k20' 'k5' 'k50' 'k60' 'k7' 'k70' 'k80' 'k9' 'lightning'
 'l型接口' 'magic6' 'max' 'mi' 'mini' 'mirco' 'note13' 'note7' 'op' 'pd'
 'plus' 'pro' 'r9' 'reno10' 'reno11' 'reno12' 'reno9' 'to' 'tpc' 'typec'
 'typec转typec' 't型接口' 'usb' 'usb口' 'v' 'v120' 'v33' 'v3380' '

In [500]:
# 尝试嵌入法，下过变差了，threshold=0.0005 差不多，不能过大，缩减到96个特征  1/219 ,每个特征的重要性大概 0.005
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel

from sklearn.tree import DecisionTreeClassifier
print(X_var.shape)
embedded_mode = SelectFromModel(DecisionTreeClassifier(criterion="entropy"), threshold=0.0004)
X_embedded = embedded_mode.fit_transform(X, y)
print(X_embedded.shape,vectorizer.get_feature_names_out()[embedded_mode.get_support()])
# X_embedded = VarianceThreshold().fit_transform(X_embedded)


# 方差虽然变大了，但是特征少了
# X_pd = pd.DataFrame(X_embedded.toarray())
# X_pd.var().mean(),X_pd.var().median(),X_pd.var().min(),X_pd.var().max(),X_pd

(4493, 241)
(4493, 81) ['.' '1' '1.5米' '100w' '120w' '12w' '150' '150w' '18w' '1米' '200w' '240w'
 '25w' '29w' '2米' '30w' '33w' '40w' '44w' '45' '45w' '55w' '5a' '65w'
 '66w' '67w' '6a' '80w' '88w' '90w' 'a' 'c' 'cc' 'ctoc口' 'c口' 'd' 'g' 'ic'
 'ipad' 'l型接口' 'max' 'mi' 'op' 'pd' 'typec' 't型接口' 'usb' 'vo' '一加' '三星'
 '主推' '充' '充电头' '全兼容' '加' '努比亚' '华' '双c' '双口' '双口头' '大壳头g' '头' '安卓' '小布丁'
 '小米' '弯头' '数据线' '白' '笔记本' '米' '紫口' '线' '绿口' '编织' '胶囊' '苹果' '荣' '荣max'
 '金标' '金标线' '黄口']


In [208]:
# 尝试包装法
# 包装法 , 逻辑回归优先使用嵌入法，svm 优先使用包装法
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

rfc = DecisionTreeClassifier(criterion="entropy",random_state=42)
rfe = RFE(rfc, n_features_to_select=80)
X_rfe = rfe.fit_transform(X, y)

In [9]:
import cutword

import cutword

cutter = cutword.Cutter(
    custom_dict_path="userdict.txt",
)
res = cutter.cutword("华为主推G")
print(res)



Loading dictionaries from D:\ProgramData\miniconda3\envs\p\Lib\site-packages\cutword\dict.txt
Loading dictionaries from userdict.txt
['华为', '主推', 'G']


In [16]:
# 尝试结果
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import cross_val_score,KFold
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
# 假设你有 X (特征), y (标签)
skf = MultilabelStratifiedKFold(n_splits=5)

md = MultiOutputClassifier(RandomForestClassifier(n_estimators=10,criterion="entropy"))


md.fit(X_var,y)

X_input = vectorizer.transform(["华66双口头【66W快充头+1米线】"])
n_pred = md.predict(X_input)
predicted_codes = mlb.inverse_transform(n_pred)
predicted_codes


[(np.str_('华66w双口头【A+C】-25w'), np.str_('华6a线-1米'))]

In [517]:
# 持久化
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import cross_val_score,KFold
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import joblib
# 假设你有 X (特征), y (标签)
skf = MultilabelStratifiedKFold(n_splits=5)

md = MultiOutputClassifier(DecisionTreeClassifier(criterion="log_loss",max_depth=10))

# X_embedded  X_var
print(X_embedded.shape)
md.fit(X_embedded,y)
joblib.dump(md, "model.bin")



(4493, 81)


['model.bin']

In [ ]:
import joblib
md = joblib.load("model.bin")
X_input = vectorizer.transform(["华 充电头 数据线 华双口66w 充电头+1米线"])
print(X_input.shape)
X_input = X_input[:,embedded_mode.get_support()]
print(X_input.shape)
n_pred = md.predict(X_input)
predicted_codes = mlb.inverse_transform(n_pred)
predicted_codes

(1, 242)
(1, 81)


[(np.str_('华66w双口头【A+C】-25w'), np.str_('华6a线-1米'))]